In [2]:
import pymysql  
  
# 创建连接  
connection = pymysql.connect(  
    host='localhost',  # 数据库服务器地址  
    user='root',   # 数据库用户名  
    password='root', # 数据库密码  
    database='bilibili',  # 数据库名  
    charset='utf8mb4',  # 字符集  
    cursorclass=pymysql.cursors.DictCursor  # 返回字典形式的游标  
)  
  
try:  
    with connection.cursor() as cursor:  
        # 在此处执行 SQL 查询  
        sql = "SELECT `id`, `username` FROM `test` "  
        cursor.execute(sql)  
        result = cursor.fetchone()  
        print(result)  
finally:  
    connection.close()

{'id': 1, 'username': 'bob'}


In [ ]:
import pymysql  
  
# 创建连接  
connection = pymysql.connect(  
    host='gateway01.ap-northeast-1.prod.aws.tidbcloud.com',  # 数据库服务器地址 
    port='4000', # 端口号
    
    user='QozzwDAVqSfD5w1.root',   # 数据库用户名  
    password='3536XBwQcinSnsix', # 数据库密码  
    database='test',  # 数据库名  
    charset='utf8mb4',  # 字符集  
    cursorclass=pymysql.cursors.DictCursor  # 返回字典形式的游标  
)  
  
try:  
    with connection.cursor() as cursor:  
        # 在此处执行 SQL 查询  
        sql = "SELECT `id`, `username` FROM `test` "  
        cursor.execute(sql)  
        result = cursor.fetchone()  
        print(result)  
finally:  
    connection.close()